In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

In [ ]:
pd.options.display.max_columns = 29
pd.options.display.max_rows = 29
df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
columns = []
for i in df.columns:
    if df[i].isnull().sum() > 0 :
        columns.append(i)
        print(i)
        print(df[i].unique())

In [ ]:
bool_col = ['surgery', 'surgical_lesion', 'cp_data']
for i in bool_col:
    df[i].replace('yes', int('1'), inplace = True)
    df[i].replace('no', int('0'), inplace = True)

In [ ]:
outcome = {'died': 0, 'euthanized': 1, 'lived': 2}
df['outcome'].replace(outcome, inplace = True)

In [ ]:
age = {'young': 1, 'adult': 2}
df['age'].replace(age, inplace = True)

In [ ]:
columns

In [ ]:
for i in columns:
    df[i].fillna(df[i].mode().iloc[0], inplace = True)
df.isnull().sum()

In [ ]:
temp = {
    'cold': 0,
    'cool': 1,
    'normal': 2,
    'warm': 3
}
ppulse = {
    'absent': 0,
    'reduced': 1,
    'normal': 2,
    'increased': 3
}
memb = {
    'normal_pink': 1,
    'pale_pink': 2,
    'bright_pink': 3,
    'bright_red': 4,
    'pale_cyanotic': 5,
    'dark_cyanotic': 6
}
time = {
    'less_3_sec': 1,
    '3': 3,
    'more_3_sec': 5
}
pain = {
    'mild_pain': 1,   
    'depressed': 2,
    'moderate': 3,
    'alert': 4,
    'severe_pain': 5,
    'extreme_pain': 6,
    'slight': 7
}
peri = {
    'normal': 1,
    'hypermotile': 2,
    'hypomotile': 3,
    'distend_small': 4,
    'absent': 5
}
dist = {
    'none': 0,
    'slight': 1,
    'moderate': 2,
    'severe': 3
}
tube = {
    'none': 0,
    'slight': 1,
    'significant': 2
}
refl = {
    'none': 0,
    'less_1_liter': 0.5,
    'slight': 1,
    'more_1_liter': 2
}
rect = {
    'absent': 0,
    'decreased': 1,
    'normal': 2,
    'increased': 3,
    'serosanguious': 4
}
abdm = {
    'firm': 1,
    'other': 2,
    'normal': 3,
    'distend_small': 4,
    'distend_large': 5}
look = {
    'clear': 0,
    'cloudy': 1,
    'serosanguious': 2
}

In [ ]:
objects = [
    ppulse,
    memb,
    time,
    pain,
    peri,
    dist,
    tube,
    refl,
    rect,
    abdm,
    look
]
obj_arr = np.array(temp)
for i in objects:
    now = np.array(i)
    obj_arr = np.append(obj_arr, now)
obj_arr

In [ ]:
j = 0
for i in columns:
    df[i].replace(obj_arr[j], inplace = True)
    j += 1

In [ ]:
y_train = df['outcome']
y_train.head()

In [ ]:
x_train = df.drop(   
    columns = [
        'id', 
        'hospital_number',
        'outcome',
        'rectal_exam_feces',
        'surgical_lesion'
    ]
)
x_train.head()

In [ ]:
ndf = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
ndf.head()

In [ ]:
ndf.isnull().sum()

In [ ]:
bool_col = ['surgery', 'surgical_lesion', 'cp_data']
for i in bool_col:
    ndf[i].replace('yes', int('1'), inplace = True)
    ndf[i].replace('no', int('0'), inplace = True)

In [ ]:
age = {'young': 1, 'adult': 2}
ndf['age'].replace(age, inplace = True)

In [ ]:
for i in columns:
    ndf[i].fillna(ndf[i].mode().iloc[0], inplace = True)
ndf.isnull().sum()

In [ ]:
j = 0
for i in columns:
    ndf[i].replace(obj_arr[j], inplace = True)
    j += 1

In [ ]:
x_test = ndf.drop(
    columns = [
        'id', 
        'hospital_number',
        'rectal_exam_feces',
        'surgical_lesion'
    ]
)
x_test.head()

In [ ]:
model = LGBMClassifier(n_estimators = 100,
                       max_depth = 2,
                       random_state = 42)

model.fit(x_train, y_train)
y_test = model.predict(x_test).astype(str)

In [ ]:
outcome = {
    '0': 'died',
    '1': 'euthanized',
    '2': 'lived'
}

In [ ]:
rdf = pd.DataFrame(y_test)
rdf.columns = ['outcome']
rdf['outcome'].replace(outcome, inplace = True)

In [ ]:
key = np.arange(1235, 2059).astype(int)
key = pd.DataFrame(key)
key.columns = ['id']
submission = pd.concat([key, rdf], axis = 1)
submission.to_csv('/kaggle/working/submission.csv', index = False, header = True)